# Quiz Notebook 3: FEI Protocol PCV Management

This Quiz Notebook is associated to [**Analysis Notebook 3: PCV Management**](../3_protocol_controlled_value.ipynb).

The goal of this notebook is to understand how to do A/B testing with pre-implemented policies.

# Quiz Setup 

- The first section of all quiz notebooks contains all necessary code to run the relevant simulation **within the quiz**.
- However, this is done for enabling experimentation. **In practice, each notebook should be gone through with the associated analysis notebook**.
- Analysis notebook sections relevant to quiz questions will be pointed to in the analysis notebooks directly.
- Solution cells are not meant to directly execute, as these may only contain the relevant snippets.

# Experiment Setup

We begin with several experiment-notebook-level preparatory setup operations:

* Import relevant dependencies
* Import relevant experiment templates
* Create copies of experiments
* Configure and customize experiments 

Analysis-specific setup operations are handled in their respective notebook sections.

In [ ]:
# Import the setup module:
# * sets up the Python path
# * runs shared notebook configuration methods, such as loading IPython modules
import setup

import copy
import logging
import numpy as np
import pandas as pd
import plotly.express as px

import experiments.notebooks.visualizations as visualizations
from experiments.run import run
from experiments.utils import display_code

In [ ]:
import plotly.io as pio
png_renderer = pio.renderers["png"]
png_renderer.width = 1200
png_renderer.height = 500
# png_renderer.scale = 1

pio.renderers.default = "png"

In [ ]:
from operator import lt, gt

In [ ]:
# Enable/disable logging
logger = logging.getLogger()
logger.disabled = False

In [ ]:
# Import experiment templates
import experiments.default_experiment as default_experiment

In [ ]:
# Inspect experiment template
display_code(default_experiment)

### Goal: Extend Simulation by A/B Testing Alternative Policies

#### Prompt:

In [ ]:
parameter_overrides = {
    "target_rebalancing_condition": [gt, lt], # Simulate decrease and increase of stable backing
    "target_stable_pcv_ratio": [None], # Disable Stable PCV Ratio Targeting
    "target_stable_backing_ratio": [0.3, 0.8], # Simulate decrease and increase of stable backing
    "rebalancing_period": [int(365/4)],  # Quarterly rebalancing
    "yield_withdrawal_period": [None],  # Disable yield-withdrawal policy
    "yield_reinvest_period": [None],  # Disable yield-reinvestment policy
}

In [ ]:
# Create a simulation for each analysis
simulation_1 = copy.deepcopy(default_experiment.experiment.simulations[0])

In [ ]:
# Experiment configuration
simulation_1.model.runs = 1

# Override inital state
simulation_1.model.initial_state.update({})

# Override default experiment System Parameters
simulation_1.model.params.update(parameter_overrides)

In [ ]:
# Analysis-specific setup

In [ ]:
# Experiment execution
# df, exceptions = run(simulation_1)

#### Questions:

1. **T/F** the FEI Protocol model has **two** different Policies for PCV management and rebalancing.

**A**: True.

2. Which state variables do each of these policies target, and where are the files for these policies located?

**A:** The two policies, found in **/model/parts/pcv_management** respectively target **stable_pcv_ratio** and **stable_backing_ratio**.

_BONUS:_ how is deposit prioritization calculated for rebalancing? Hint - see PCV rebalancing algorithm lucidchart document (or its export).

In [ ]:
# TODO: link

In both the case of Stable Backing and Stable PCV ratio targeting, the algorithm will initially see if the amount to be rebalanced (ex: volatile asset sold, equivalent dollar value of stable bought) can be viably performed via sale of idle PCV deposit balance. If the idle PCV deposit does not have enough balance, the yield bearing deposits will be used, which implies withdrawal of PCV yield.

### Extension: A/B Testing - PCV at Risk for Stable PCV Ratio Targeting

1. Edit **Analysis 2 - PCV at Risk for Stable Backing Ratio Targets** in the following way: 

    Keeping the same levels and directions of rebalancing in the parameter sweep, **configure the parameter overrides to target stable PCV ratio instead of stable backing ratio**.
    
    Set the number of monte carlo runs to 25.

In [ ]:
parameter_overrides = {
    "target_rebalancing_condition": [gt, lt], # Simulate decrease and increase of stable backing
    "target_stable_pcv_ratio": [0.3, 0.8], # Simulate decrease and increase of stable PCV
    "target_stable_backing_ratio": [None], # Disable targeting of Stable Backing Ratio
    "rebalancing_period": [int(365/4)],  # Quarterly rebalancing
    "yield_withdrawal_period": [None],  # Disable yield-withdrawal policy
    "yield_reinvest_period": [None],  # Disable yield-reinvestment policy
}

2. Run the experiment and compare all metrics related to PCVaR which are outputs of the analysis in analyses 3 vs 4.

3. Which policy targeting (Stable Backing vs Stable PCV) provides better risk management for the protocol, across all monte carlo runs?


**A:** PCV at Risk delta between the volatile-exposed policy vs the conservative policy is greater with the Stable backing ratio than stable PCV ratio. The delta in quantiles is also slightly greater when targeting stable backing.

In terms of effect on the collateral ratio, the empirical probability of the conservative policy being an effective risk management setting is unchanged between settings, though visually we notice greater divergence in collateral ratio when targeting stable PCV ratio.

While we cannot conclude directly which policy is better in absolute terms, policy selection can be informed by a-priori governance goals in KPI setting. However, by construction the stable backing ratio depends on two volatile processes - total PCV and user-circulating FEI, whereas stable PCV targeting only depends on total PCV.